*Thanks to Viet Nguyen MD for this book*


**Use Case**
As a physician, I want to calculate a patient's cardiovascular risk using the Framingham calculator based on LDL 
    (http://reference.medscape.com/calculator/framingham-coronary-risk-ldl)


**Define Functional requirements**  
Step 1 - Define required clinical data elements  
- Patient Age  
- Patient Gender  
- LDL Cholesterol in mg/dL  
- HDL Cholesterol in mg/dL  
- Blood Pressure  
- Diabetes  
- Smoking Status  
    
    
Step 2 - Identify FHIR Resources to support use case data (http://hl7.org/fhir/resourcelist.html)
- Patient Age (http://hl7.org/fhir/patient.html) 
- Patient.birthDate minus today's date
- Patient Gender - Patient.contact.gender
- LDL Cholesterol in mg/dL - Observation resource (http://www.hl7.org/fhir/diagnosticreport-example-lipids.html)
- HDL Cholesterol in mg/dL - Observation resource
- Blood Pressure - Observation resource (http://hl7.org/fhir/observation-vitalsigns.html or http://www.hl7.org/fhir/observation-example-bloodpressure.html)
- Diabetes - Condition resource (http://hl7.org/fhir/condition.html)
- Smoking Status - Observation resource (http://hl7.org/fhir/us/core/2017Jan/ValueSet-us-core-observation-ccdasmokingstatus.html)
    

Step 3 - Identify data elements within each FHIR Resource definition necessary to support use case data


## Initialize the environment to run the application

In [3]:
# Install the Python FHIR client package in the current Jupyter kernel using pip
#   Library location in github: https://github.com/smart-on-fhir/client-py
import sys
!{sys.executable} -m pip install fhirclient

In [4]:
#import the client library that understands how to make FHIR calls and interpret the results
#we can then use it to interact with the server and manipulate the resources
from fhirclient import client
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import requests
import json
print('FHIR client library has been loaded')

FHIR client library has been loaded


## Load the data needed for the Framingham analysis

In [5]:
#load the Patient from the database, and display demographics
from dateutil.relativedelta import relativedelta  # a utility function for comparing dates
import datetime
# Define the FHIR Endpoint. We define the name of this app (used in SMART calls), and the location of the server
settings = {
    'app_id': 'my_web_app',
    'api_base': 'http://hapi.fhir.org/baseStu3/'
}

In [6]:
# Create an 'instance' of the FHIR library that points to the FHIR server. We use this to make the calls.
db = client.FHIRClient(settings=settings)

In [29]:
# Perform a GET (read) on the patient with an id of 'cf-1495289345197' (a test patient we created)
# We could also search on name or any other property to find the patient
# Under the hood, a GET request is issued to the server, eg GET [server]/Patient/cf-1495289345197
import fhirclient.models.patient as p
print('Reading patient, please wait...')

# patient = p.Patient.read('cf-1533147316668', db.server)

search = p.Patient.where(struct={'identifier': 'https://github.com/synthetichealth/synthea|0166a3a8-2fa5-4ebb-a831-f03dc454f314'})
patients = search.perform_resources(db.server)
patient = patients[0]
patient_id = patient.id


Reading patient, please wait...


In [13]:
# Print the patient name. A Patient can have more than one name (ie Patient.name is multiple)
for name in patient.name:
    first_name = name.given[0]
    last_name = name.family
    print ('Patient\'s Name(s) = ',first_name,last_name)

Patient's Name(s) =  Test954 Test821
Patient's Name(s) =  Suzie954 Kuhn492


In [14]:
# Print patient's gender (only 1 gender !)
gender = patient.gender
print ('Gender = ',gender)

Gender =  male


In [15]:
# Define variable DOB as the patient's date of birth in a string format (so it can be displayed)
DOB = patient.birthDate.isostring
print ('DOB = ', DOB)

DOB =  1968-06-20


In [16]:
# Define and print today variable. We'll use this for the age (coming up next)
now = datetime.datetime.today()
print ('Today\'s Date = ', now)

Today's Date =  2020-01-23 13:20:17.203569


In [17]:
#now get the patients age using the relativedelta library function. Don't sweat the details!
#Define DOB2 date object
DOB2 = datetime.datetime.strptime(DOB, '%Y-%m-%d')

In [18]:
# Calcuate patient's age using rdelta method of the dateutil module
rdelta = relativedelta(now, DOB2)
age=rdelta.years

# print the years from the rdelta object
print ('Patient\'s age = ',age)

Patient's age =  51


In [19]:
#All of the above scripts could have been combined into a single cell
#Below are all the print statements:
print ('Patient\'s Name(s) = ',first_name,last_name)
print ('Gender = ',gender)
print ('DOB = ', DOB)
print ('Today\'s Date = ', now)
print ('Patient\'s age = ',age)

Patient's Name(s) =  Suzie954 Kuhn492
Gender =  male
DOB =  1968-06-20
Today's Date =  2020-01-23 13:20:17.203569
Patient's age =  51


In [39]:
# Get the most recent LDL. This is an Observation where the Observation.code is LOINC 18262-6
import fhirclient.models.observation as obs   #allows search of Observation resources
# specify the search, setting the patient, the code we want and in descending order by date
search = obs.Observation.where(struct={'code':"18262-6",'_sort':'-date'})
#show the actual url that is generated
url = search.construct()
print('Query: ', url)

LDLBundle = search.perform_resources(db.server)  #perform_resources() will place the returned resources into an array
if LDLBundle:
    #As the search is in date descending order, the first one will be the most recent
    LDL_val = LDLBundle[0].valueQuantity.value
    LDL_unit = LDLBundle[0].valueQuantity.unit
    print('The most recent value is ',str(LDL_val) + " " + LDL_unit)
    print('Referenced patient: ', LDLBundle[0].subject.reference)

Query:  Observation?code=18262-6&_sort=-date
The most recent value is  149.43607005147118 mg/dL
Referenced patient:  Patient/2000800


In [30]:
# Get the most recent LDL. This is an Observation where the Observation.code is LOINC 18262-6
import fhirclient.models.observation as obs   #allows search of Observation resources
# specify the search, setting the patient, the code we want and in descending order by date
search = obs.Observation.where(struct={'patient':patient_id,'code':"18262-6",'_sort':'-date'})

#show the actual url that is generated
url = search.construct()
print('Query: ', url)

LDLBundle = search.perform_resources(db.server)  #perform_resources() will place the returned resources into an array
if LDLBundle:
    #As the search is in date descending order, the first one will be the most recent
    LDL_val = LDLBundle[0].valueQuantity.value
    LDL_unit = LDLBundle[0].valueQuantity.unit
    print('The most recent value is ',str(LDL_val) + " " + LDL_unit)

Query:  Observation?patient=1035907&code=18262-6&_sort=-date
The most recent value is  82 mg/dL


In [40]:
# Find the most recent HDL using a GET on Observation, specifying the patient id and the code for HDL
search = obs.Observation.where(struct={'patient':patient_id,'code':"2085-9",'_sort':'-date'})

#show the actual url that is generated
print('Query: ', search.construct())

HDL = search.perform_resources(db.server)
if HDL:
    HDL_val = HDL[0].valueQuantity.value
    print('The most recent value is ',str(HDL_val) + " " + HDL[0].valueQuantity.unit)

Query:  Observation?patient=1035907&code=2085-9&_sort=-date
The most recent value is  74 mg/dL


In [41]:
# Find the most recent Blood Pressure
search = obs.Observation.where(struct={'patient':patient_id,'code':"55284-4",'_sort':'-date'})
print('Query: ', search.construct())
press = search.perform_resources(db.server)
if press:
    #Blood pressure has both values in a single Observation, in different components
    #We assume that the first component is the Systolic, but should really check the code to be sure
    sys = press[0].component[0].valueQuantity.value
    dia = press[0].component[1].valueQuantity.value
    print("Systolic: "+str(sys)+" "+press[0].component[0].valueQuantity.unit)
    print("Diastolic: "+str(dia)+" "+press[0].component[0].valueQuantity.unit)

Query:  Observation?patient=1035907&code=55284-4&_sort=-date
Systolic: 101 mmHg
Diastolic: 84 mmHg


In [47]:
# Does the patient have diabetes? Check if there is a Condition with a code of either Type 1 or Type 2 Diabates
# We assume a SNOMED code, note that the system for SNOMED is specified - eg http://snomed.info/sct|44054006
# In practice, you'd need to check for multiple codes - there are terminology services that make this easier
from fhirclient import client
import fhirclient.models.condition as cond
#This search is for type 1 diabetes
search1 = cond.Condition.where(struct={'patient':patient_id,'code':'http://snomed.info/sct|46635009'})
print('Query: ', search1.construct())
#This is for type 2 diabetes
search2 = cond.Condition.where(struct={'patient':patient_id,'code':'http://snomed.info/sct|44054006'})
print('Query: ', search2.construct())

#perform the query
type1 = search1.perform_resources(db.server)
type2 = search2.perform_resources(db.server)

if type1:
    print('Patient has type 1 diabetes')
    
if type2:
    print('Patient has type 2 diabetes')

#condition that says whether they have one of these two types of diabetes
if type1 or type2:
    diabetes = True   #this is the value that will be used in the Framingham calculation
    print("Patient has diabetes")
else:
    diabetes = False
    print("Patient does not have diabetes")


Query:  Condition?patient=1035907&code=http%3A%2F%2Fsnomed.info%2Fsct%7C46635009
Query:  Condition?patient=1035907&code=http%3A%2F%2Fsnomed.info%2Fsct%7C44054006
Patient does not have diabetes


In [48]:
#Look for an Observation with a LOINC code for smoker
from fhirclient import client
import fhirclient.models.observation as obs
search = obs.Observation.where(struct={'patient':'Patient/cf-1533147316668','code':'http://loinc.org|72166-2'})
smoking = search.perform_resources(db.server)
if smoking:
    smoke = True
    print("Patient smokes")
else:
    smoke = False
    print("Patient doesn't smoke")


Patient doesn't smoke


## Now that we have the data we need, we can perform the Framingham calculations

In [49]:
points = 0

#this block is for a Male
if gender == "male":
    if age <= 34:
        points += -1
    if age >= 40 and age <= 44:
        points += 1
    if age >= 45 and age <= 49:
        points += 2
    if age >= 50 and age <= 54:
        points += 3
    if age >= 55 and age <= 59:
        points += 4
    if age >= 60 and age <= 64:
        points += 5
    if age >= 65 and age <= 69:
        points += 6
    if age >= 70:
        points += 7
    if LDL_val < 100:
        points += -3
    if LDL_val >= 160 and LDL_val <= 189:
        points += 1
    if LDL_val >= 190:
        points += 2
    if HDL_val < 35:
        points += 2
    if HDL_val >= 35 and LDL_val <= 44:
        points += 1
    if HDL_val >= 60:
        points += -1
    if sys <=129 and dia >= 85 and dia <= 89:
        points += 1
    if sys >= 130 and sys <=139 and dia < 90:
        points += 1
    if sys <= 139 and dia >= 90 and dia <= 99:
        points += 2
    if sys >= 140 and sys <=159 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 2
    if smoking == True:
        points += 2
        
    print("Male analysis complete")

Male analysis complete


In [50]:
# and this for a female
if gender == "female":
    if age <= 34:
        points += -9
    if age >= 35 and age <= 39:
        points += -4
    if age >= 45 and age <= 49:
        points += 3
    if age >= 50 and age <= 54:
        points += 6
    if age >= 55 and age <= 59:
        points += 7
    if age >= 60:
        points += 8
    if LDL_val < 100:
        points += -2
    if LDL_val >= 160:
        points += 2
    if HDL_val < 35:
        points += 5
    if HDL_val >= 35 and HDL_val <= 44:
        points += 2
    if HDL_val >= 45 and HDL_val <= 49:
        points += 1
    if HDL_val >= 60:
        points += -2
    if sys < 120 and dia < 80:
        points += -3
    if sys <=159 and dia >=90 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 4
    if smoking == True:
        points += 2
    print("Female analysis complete")

In [51]:
#this table converts from points to a percentage risk
male_results = {0:3,1:4,2:4,3:6,4:7,5:9,6:11,7:14,8:18,9:22,10:27,11:33,12:40,13:47}
female_results = {0:2,1:2,2:3,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:11,11:13,12:15,13:17,14:20,15:24,16:27}

if points >=14 and gender == "male":
    risk = ">= 56%"
elif points >=17 and gender == "female":
    risk = ">= 32%"
elif gender == "male":
    percent = male_results[points]
    risk = str(percent)+"%"
else:
    percent = female_results[points]
    risk = str(percent)+"%"
print("Total number of points: "+str(points))
print("Risk of Heart Disease: "+risk)

Total number of points: 2
Risk of Heart Disease: 4%


# Now...How would you find a patient who's likely to have a higher score in the FHIR server?